In [1]:
import pandas as pd
from nameparser import HumanName
import numpy as np

In [3]:
new_cpt_data = pd.read_csv("../../Data/CPT_Panel/Panel Member Evaluations 2022_raw data.csv")
cpt_data = pd.read_csv("../../Data/CPT_Panel/Panel Member Evaluations 2022_raw data.csv", header = 1)
cpt_col = pd.read_csv("../../Data/CPT_Panel/Panel Member Evaluations 2022_raw data.csv").columns

In [4]:
dicto ={}
index = 0
for col in cpt_col:
    index += 1
    if "Unnamed" in col:
        new_col = f'Column{index}'
        dicto[col] = new_col

In [5]:
names = [x for x in cpt_col if "Unnamed" not in x]

In [6]:
index = 0
cols = [0,1,2,3,4,5,6,7]
dict_list = []
for name in names:
    peer = HumanName(name).last
    data = cpt_data.iloc[:, cols]
    for row in data[2:].itertuples():
        row_list = list(row)
        reviewer = HumanName(row.Q1).last
        question_number = 2
        for question_column in cols[1:]:
            if row_list[question_number] == "N/A - I don't know":
                rating = None
            elif isinstance(row_list[question_number], float):
                rating = row_list[question_number]
            elif peer == reviewer:
                rating = None
            else:
                rating = row_list[question_number].split(' -')[0]
            new_dict = {
            'Peer': peer,
            'Reviewer': reviewer,
            'Question': f'Q{question_number - 1}',
            'Rating': rating
        }
            question_number+=1
            dict_list.append(new_dict)
    cols = [(x + 7) if x != 0 else x for x in cols]

In [7]:
overall = pd.DataFrame(dict_list)

In [8]:
numerical = overall[overall.Question!='Q7'].dropna(subset=['Rating'])
numerical['Rating'] = numerical.Rating.astype(float)

In [9]:
#Stats
standard_dev = numerical.Rating.std()
mean = numerical.Rating.mean()
mean_plus = mean + standard_dev
mean_minus = mean - standard_dev

In [10]:
np.std(numerical.Rating)

0.970731339930765

In [11]:
standard_dev

0.9711581288456168

In [12]:
mean_plus

4.947432294047726

In [13]:
mean_minus

3.005116036356492

In [27]:
mean

3.976274165202109

In [14]:
#Average ratings
peer_average = numerical.groupby('Peer').mean()

In [15]:
#Average per question/peer
peer_question_average = numerical.groupby(['Peer', 'Question']).mean()

In [16]:
#Comments
comments = overall[overall.Question=='Q7'].dropna(subset=['Rating']).drop(columns='Question')

In [17]:
#Median per question
median = numerical.groupby(['Peer', 'Question']).median()

In [18]:
questions = ['Q1','Q2','Q2','Q3','Q4','Q5','Q6']
for question in questions:
    print(np.std(numerical[numerical.Question==question].Rating))

0.9631570050744718
0.9389353699547404
0.9389353699547404
1.0292383062855364
0.9576385535459412
1.0260185620889741
0.8783746069379378


In [142]:
for question in questions:
    print(np.mean(numerical[numerical.Question==question].Rating))

4.1103896103896105
4.123376623376624
4.123376623376624
4.046052631578948
4.14
4.033557046979865
4.222222222222222


In [19]:
#Average per question
qpr_rating = numerical.groupby(['Question']).mean()

In [20]:
peer_question_average.groupby(['Question']).mean()

,Rating
Question,
Q1,3.972527
Q2,3.963736
Q3,3.915185
Q4,3.987485
Q5,3.912393
Q6,4.098024


In [21]:
peer_question_average.groupby(['Question']).std()

,Rating
Question,
Q1,0.549462
Q2,0.393011
Q3,0.536103
Q4,0.564166
Q5,0.580881
Q6,0.441471


In [22]:
#Peer review rating
peer_rev_rating = numerical.groupby(['Peer', 'Reviewer']).mean()
numerical.groupby(['Peer', 'Reviewer']).mean()['Rating'].mean()
numerical.groupby(['Peer', 'Reviewer']).mean()['Rating'].std()

0.9020967339088961

In [23]:
#Reviewer question rating
rev_q_rating = numerical.groupby(['Reviewer', 'Question']).mean()

In [24]:
#Reviewer Average rating
review_avg_rating = numerical.groupby(['Reviewer']).mean()

In [25]:
with pd.ExcelWriter('../../Data/CPT_Panel/transformed_2022-07-20.xlsx') as writer:  
        new_cpt_data.to_excel(writer, sheet_name='Raw', index=False)
        overall.to_excel(writer, sheet_name='Overall', index=False)
        numerical.to_excel(writer, sheet_name='Rating(2)', index=False)
        numerical.to_excel(writer, sheet_name='Rating', index=False)
        numerical.to_excel(writer, sheet_name='Original_Rating', index=False)
        comments.to_excel(writer, sheet_name='Comments', index=False)
        peer_question_average.to_excel(writer, sheet_name='QPRRating')
        median.to_excel(writer, sheet_name='Median')
        peer_rev_rating.to_excel(writer, sheet_name='PeerRevRating')
        rev_q_rating.to_excel(writer, sheet_name='RevQRating')
        review_avg_rating.to_excel(writer, sheet_name='ReviewerAvgRating')
        peer_average.to_excel(writer, sheet_name='PeerAvgRating')

In [26]:
names

['Linda Barney',
 'Aaron Bossler',
 'Daniel Buffington',
 'Chris Jagmin',
 'David Kanter',
 'Barbara Levy',
 'Doug Morrow',
 'Dan Nagle',
 "Judith O'Connell",
 'Robert Piana',
 'Dan Picus',
 'Larry Simon',
 'Tim Swan',
 'Kevin Vorenkamp',
 'Gloria Wilder']